In [2]:
from IPython.display import HTML
import matplotlib.pyplot as plt
import os
import time
import sys                                     
from pathlib import Path
sys.path.insert(0, str(Path().resolve().parent))
from demoTools.demoutils import *
from openvino.inference_engine import IEPlugin, IENetwork
import cv2
# For labeling the image
from out_process import placeBoxes


### Create an Intermediate Representation of the Model
Model Optimizer creates the Intermediate Representation of the model which is the device-agnostic, generic optimization of the model. Caffe*, TensorFlow*, MXNet*, ONNX*, and Kaldi* models are supported by Model Optimizer.

We will use the YOLOv3 model. Download the model. All YOLO models are originally implemented in the Darknet framework and consists of two files: .cfg file with model configurations and .weights file with model weights.

In [2]:
! pip3 install tensorflow==1.12

  Using cached https://files.pythonhosted.org/packages/b1/ad/48395de38c1e07bab85fc3bbec045e11ae49c02a4db0100463dd96031947/tensorflow-1.12.0-cp35-cp35m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/73/fb/00a976f728d0d1fecfe898238ce23f502a721c0ac0ecfedb80e0d88c64e9/six-1.12.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/d1/4f/950dfae467b384fc96bc6469de25d832534f6b4441033c39f914efd13418/astor-0.8.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/71/e3/19762fdfc62877ae9102edf6342d71b28fbfd9dea3d2f96a882ce099b03f/Keras_Applications-1.0.8-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/28/6a/8c1f62c37212d9fc441a7e26736df51ce6f0e38455816445471f10da4f0a/Keras_Preprocessing-1.1.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/42/69/4acc9bdf349db7258867b8af96112661fbce65034de1326d65ad14c96e50/grpcio-1.24.0-cp35-cp35m-manylinux1_x86_64.whl
  Using cached h

In [2]:
! git clone https://github.com/mystic123/tensorflow-yolo-v3.git

fatal: destination path 'tensorflow-yolo-v3' already exists and is not an empty directory.


In [3]:
! cd tensorflow-yolo-v3

### Download coco.names file from the Darknet website or use labels that fit your task

In [3]:
! wget https://raw.githubusercontent.com/pjreddie/darknet/master/data/coco.names

--2019-10-01 00:41:19--  https://raw.githubusercontent.com/pjreddie/darknet/master/data/coco.names
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.188.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.188.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 625 [text/plain]
Saving to: ‘coco.names.6’

coco.names.6        100%[===================>]     625  --.-KB/s    in 0s      

2019-10-01 00:41:20 (57.6 MB/s) - ‘coco.names.6’ saved [625/625]



### Download the yolov3.weights. 

In [4]:
! wget https://pjreddie.com/media/files/yolov3.weights

--2019-10-01 00:41:25--  https://pjreddie.com/media/files/yolov3.weights
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248007048 (237M) [application/octet-stream]
Saving to: ‘yolov3.weights’

yolov3.weights      100%[===================>] 236.52M  1.14MB/s    in 3m 2s   

2019-10-01 00:44:27 (1.30 MB/s) - ‘yolov3.weights’ saved [248007048/248007048]



### Run a converter to freeze the graph. 

In [5]:
!  python3 tensorflow-yolo-v3/convert_weights_pb.py --class_names coco.names --data_format NHWC --weights_file yolov3.weights

/home/u28225/.local/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/u28225/.local/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/u28225/.local/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/u28225/.local/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:526: 

## Optimize a deep-learning model using the Model Optimizer (MO) 

In this section, you will use the Model Optimizer to convert a trained model to two Intermediate Representation (IR) files (one .bin and one .xml). The Inference Engine requires this model conversion so that it can use the IR as input and achieve optimum performance on Intel® hardware.



### 1. Create a directory to store IR files

In [8]:
!cd tensorflow-yolo-v3/
! mkdir -p FP32
! mkdir -p FP16

### 2. Run the Model Optimizer on the frozen YOLOv3 TensorFlow* model. This step generates one .xml file and one .bin file and place both files in the tutorial samples directory (located here: /object-detection/tensorflow-yolo-v3/FP32/)

In [9]:
!python3 mo_tf.py --input_model frozen_darknet_yolov3_model.pb --batch 1 --tensorflow_use_custom_operations_config /opt/intel/openvino/deployment_tools/model_optimizer/extensions/front/tf/yolo_v3.json -o tensorflow-yolo-v3/FP16 --data_type FP16

Model Optimizer arguments:
Common parameters:
	- Path to the Input Model: 	/home/u28225/Reference-samples/smart-video-workshop_30sepbackup/Yolov3/frozen_darknet_yolov3_model.pb
	- Path for generated IR: 	/home/u28225/Reference-samples/smart-video-workshop_30sepbackup/Yolov3/tensorflow-yolo-v3/FP16
	- IR output name: 	frozen_darknet_yolov3_model
	- Log level: 	ERROR
	- Batch: 	1
	- Input layers: 	Not specified, inherited from the model
	- Output layers: 	Not specified, inherited from the model
	- Input shapes: 	Not specified, inherited from the model
	- Mean values: 	Not specified
	- Scale values: 	Not specified
	- Scale factor: 	Not specified
	- Precision of IR: 	FP16
	- Enable fusing: 	True
	- Enable grouped convolutions fusing: 	True
	- Move mean values to preprocess section: 	False
	- Reverse input channels: 	False
TensorFlow specific parameters:
	- Input model in text protobuf format: 	False
	- Path to model dump for TensorBoard: 	None
	- List of shared libraries with TensorFlow cu

In [10]:
!python3 mo_tf.py --input_model frozen_darknet_yolov3_model.pb --batch 1 --tensorflow_use_custom_operations_config /opt/intel/openvino/deployment_tools/model_optimizer/extensions/front/tf/yolo_v3.json -o tensorflow-yolo-v3/FP32

Model Optimizer arguments:
Common parameters:
	- Path to the Input Model: 	/home/u28225/Reference-samples/smart-video-workshop_30sepbackup/Yolov3/frozen_darknet_yolov3_model.pb
	- Path for generated IR: 	/home/u28225/Reference-samples/smart-video-workshop_30sepbackup/Yolov3/tensorflow-yolo-v3/FP32
	- IR output name: 	frozen_darknet_yolov3_model
	- Log level: 	ERROR
	- Batch: 	1
	- Input layers: 	Not specified, inherited from the model
	- Output layers: 	Not specified, inherited from the model
	- Input shapes: 	Not specified, inherited from the model
	- Mean values: 	Not specified
	- Scale values: 	Not specified
	- Scale factor: 	Not specified
	- Precision of IR: 	FP32
	- Enable fusing: 	True
	- Enable grouped convolutions fusing: 	True
	- Move mean values to preprocess section: 	False
	- Reverse input channels: 	False
TensorFlow specific parameters:
	- Input model in text protobuf format: 	False
	- Path to model dump for TensorBoard: 	None
	- List of shared libraries with TensorFlow cu

### 3. Navigate to the tutorial sample model directory¶ 

In [11]:
! cd tensorflow-yolo-v3/FP16

### 4. Verify creation of the optimized model files (the IR files)

In [12]:
! ls

cars_1900.mp4			obj_det_gpu.e57041
coco.names			obj_det_gpu.e57050
coco.names.1			obj_det_gpu.o57041
coco.names.2			obj_det_gpu.o57050
coco.names.3			object_detection_demo_yolov3_async.py
coco.names.4			object_detection_job1.sh
coco.names.5			Object_Detection_using_Yolov3.ipynb
coco.names.6			out_process.py
FP16				__pycache__
FP32				results
frozen_darknet_yolov3_model.pb	tensorflow-yolo-v3
head-pose-female-male.mp4	Untitled.ipynb
mo_tf.py			yolov3.weights


## Use the YOLOv3 model and Inference Engine in an object detection application 

###  Use the sample app (object_detection_demo_yolov3_async.py) from the Intel® Distribution of OpenVINO™ toolkit.

In [ ]:
! python3 object_detection_demo_yolov3_async.py -h

### Source your environmental variables 

In [3]:
%%bash
source /opt/intel/openvino/bin/setupvars.sh

[setupvars.sh] OpenVINO environment initialized


In [4]:
os.environ["VIDEO"] = "cars_1900.mp4"

###  Run the sample application to use the Inference Engine

In [ ]:
! python3 object_detection_demo_yolov3_async.py  -i $VIDEO -m tensorflow-yolo-v3/FP32/frozen_darknet_yolov3_model.xml -l $HOME/inference_engine_samples_build/intel64/Release/lib/libcpu_extension.so

[ INFO ] Loading network files:
	tensorflow-yolo-v3/FP32/frozen_darknet_yolov3_model.xml
	tensorflow-yolo-v3/FP32/frozen_darknet_yolov3_model.bin
[ INFO ] Preparing inputs
[ INFO ] Loading model to the plugin
To close the application, press 'CTRL+C' or any key with focus on the output window
[ INFO ] Layer detector/yolo-v3/Conv_14/BiasAdd/YoloRegion parameters: 
[ INFO ]          num     : 3
[ INFO ]          classes : 80
[ INFO ]          coords  : 4
[ INFO ]          anchors : [10.0, 13.0, 16.0, 30.0, 33.0, 23.0, 30.0, 61.0, 62.0, 45.0, 59.0, 119.0, 116.0, 90.0, 156.0, 198.0, 373.0, 326.0]
[ INFO ] Layer detector/yolo-v3/Conv_22/BiasAdd/YoloRegion parameters: 
[ INFO ]          num     : 3
[ INFO ]          classes : 80
[ INFO ]          coords  : 4
[ INFO ]          anchors : [10.0, 13.0, 16.0, 30.0, 33.0, 23.0, 30.0, 61.0, 62.0, 45.0, 59.0, 119.0, 116.0, 90.0, 156.0, 198.0, 373.0, 326.0]
[ INFO ] Layer detector/yolo-v3/Conv_6/BiasAdd/YoloRegion parameters: 
[ INFO ]          num   

[ INFO ] Layer detector/yolo-v3/Conv_6/BiasAdd/YoloRegion parameters: 
[ INFO ]          num     : 3
[ INFO ]          classes : 80
[ INFO ]          coords  : 4
[ INFO ]          anchors : [10.0, 13.0, 16.0, 30.0, 33.0, 23.0, 30.0, 61.0, 62.0, 45.0, 59.0, 119.0, 116.0, 90.0, 156.0, 198.0, 373.0, 326.0]
[ INFO ] Layer detector/yolo-v3/Conv_14/BiasAdd/YoloRegion parameters: 
[ INFO ]          num     : 3
[ INFO ]          classes : 80
[ INFO ]          coords  : 4
[ INFO ]          anchors : [10.0, 13.0, 16.0, 30.0, 33.0, 23.0, 30.0, 61.0, 62.0, 45.0, 59.0, 119.0, 116.0, 90.0, 156.0, 198.0, 373.0, 326.0]
[ INFO ] Layer detector/yolo-v3/Conv_22/BiasAdd/YoloRegion parameters: 
[ INFO ]          num     : 3
[ INFO ]          classes : 80
[ INFO ]          coords  : 4
[ INFO ]          anchors : [10.0, 13.0, 16.0, 30.0, 33.0, 23.0, 30.0, 61.0, 62.0, 45.0, 59.0, 119.0, 116.0, 90.0, 156.0, 198.0, 373.0, 326.0]
[ INFO ] Layer detector/yolo-v3/Conv_6/BiasAdd/YoloRegion parameters: 
[ INFO ]    

[ INFO ] Layer detector/yolo-v3/Conv_6/BiasAdd/YoloRegion parameters: 
[ INFO ]          num     : 3
[ INFO ]          classes : 80
[ INFO ]          coords  : 4
[ INFO ]          anchors : [10.0, 13.0, 16.0, 30.0, 33.0, 23.0, 30.0, 61.0, 62.0, 45.0, 59.0, 119.0, 116.0, 90.0, 156.0, 198.0, 373.0, 326.0]
[ INFO ] Layer detector/yolo-v3/Conv_14/BiasAdd/YoloRegion parameters: 
[ INFO ]          num     : 3
[ INFO ]          classes : 80
[ INFO ]          coords  : 4
[ INFO ]          anchors : [10.0, 13.0, 16.0, 30.0, 33.0, 23.0, 30.0, 61.0, 62.0, 45.0, 59.0, 119.0, 116.0, 90.0, 156.0, 198.0, 373.0, 326.0]
[ INFO ] Layer detector/yolo-v3/Conv_22/BiasAdd/YoloRegion parameters: 
[ INFO ]          num     : 3
[ INFO ]          classes : 80
[ INFO ]          coords  : 4
[ INFO ]          anchors : [10.0, 13.0, 16.0, 30.0, 33.0, 23.0, 30.0, 61.0, 62.0, 45.0, 59.0, 119.0, 116.0, 90.0, 156.0, 198.0, 373.0, 326.0]
[ INFO ] Layer detector/yolo-v3/Conv_6/BiasAdd/YoloRegion parameters: 
[ INFO ]    

[ INFO ] Layer detector/yolo-v3/Conv_6/BiasAdd/YoloRegion parameters: 
[ INFO ]          num     : 3
[ INFO ]          classes : 80
[ INFO ]          coords  : 4
[ INFO ]          anchors : [10.0, 13.0, 16.0, 30.0, 33.0, 23.0, 30.0, 61.0, 62.0, 45.0, 59.0, 119.0, 116.0, 90.0, 156.0, 198.0, 373.0, 326.0]
[ INFO ] Layer detector/yolo-v3/Conv_14/BiasAdd/YoloRegion parameters: 
[ INFO ]          num     : 3
[ INFO ]          classes : 80
[ INFO ]          coords  : 4
[ INFO ]          anchors : [10.0, 13.0, 16.0, 30.0, 33.0, 23.0, 30.0, 61.0, 62.0, 45.0, 59.0, 119.0, 116.0, 90.0, 156.0, 198.0, 373.0, 326.0]
[ INFO ] Layer detector/yolo-v3/Conv_22/BiasAdd/YoloRegion parameters: 
[ INFO ]          num     : 3
[ INFO ]          classes : 80
[ INFO ]          coords  : 4
[ INFO ]          anchors : [10.0, 13.0, 16.0, 30.0, 33.0, 23.0, 30.0, 61.0, 62.0, 45.0, 59.0, 119.0, 116.0, 90.0, 156.0, 198.0, 373.0, 326.0]
[ INFO ] Layer detector/yolo-v3/Conv_6/BiasAdd/YoloRegion parameters: 
[ INFO ]    

[ INFO ] Layer detector/yolo-v3/Conv_6/BiasAdd/YoloRegion parameters: 
[ INFO ]          num     : 3
[ INFO ]          classes : 80
[ INFO ]          coords  : 4
[ INFO ]          anchors : [10.0, 13.0, 16.0, 30.0, 33.0, 23.0, 30.0, 61.0, 62.0, 45.0, 59.0, 119.0, 116.0, 90.0, 156.0, 198.0, 373.0, 326.0]
[ INFO ] Layer detector/yolo-v3/Conv_14/BiasAdd/YoloRegion parameters: 
[ INFO ]          num     : 3
[ INFO ]          classes : 80
[ INFO ]          coords  : 4
[ INFO ]          anchors : [10.0, 13.0, 16.0, 30.0, 33.0, 23.0, 30.0, 61.0, 62.0, 45.0, 59.0, 119.0, 116.0, 90.0, 156.0, 198.0, 373.0, 326.0]
[ INFO ] Layer detector/yolo-v3/Conv_22/BiasAdd/YoloRegion parameters: 
[ INFO ]          num     : 3
[ INFO ]          classes : 80
[ INFO ]          coords  : 4
[ INFO ]          anchors : [10.0, 13.0, 16.0, 30.0, 33.0, 23.0, 30.0, 61.0, 62.0, 45.0, 59.0, 119.0, 116.0, 90.0, 156.0, 198.0, 373.0, 326.0]
[ INFO ] Layer detector/yolo-v3/Conv_6/BiasAdd/YoloRegion parameters: 
[ INFO ]    

...

[ INFO ] Layer detector/yolo-v3/Conv_14/BiasAdd/YoloRegion parameters: 
[ INFO ]          num     : 3
[ INFO ]          classes : 80
[ INFO ]          coords  : 4
[ INFO ]          anchors : [10.0, 13.0, 16.0, 30.0, 33.0, 23.0, 30.0, 61.0, 62.0, 45.0, 59.0, 119.0, 116.0, 90.0, 156.0, 198.0, 373.0, 326.0]
[ INFO ] Layer detector/yolo-v3/Conv_22/BiasAdd/YoloRegion parameters: 
[ INFO ]          num     : 3
[ INFO ]          classes : 80
[ INFO ]          coords  : 4
[ INFO ]          anchors : [10.0, 13.0, 16.0, 30.0, 33.0, 23.0, 30.0, 61.0, 62.0, 45.0, 59.0, 119.0, 116.0, 90.0, 156.0, 198.0, 373.0, 326.0]
[ INFO ] Layer detector/yolo-v3/Conv_6/BiasAdd/YoloRegion parameters: 
[ INFO ]          num     : 3
[ INFO ]          classes : 80
[ INFO ]          coords  : 4
[ INFO ]          anchors : [10.0, 13.0, 16.0, 30.0, 33.0, 23.0, 30.0, 61.0, 62.0, 45.0, 59.0, 119.0, 116.0, 90.0, 156.0, 198.0, 373.0, 326.0]
[ INFO ] Layer detector/yolo-v3/Conv_14/BiasAdd/YoloRegion parameters: 
[ INFO ]   

[ INFO ] Layer detector/yolo-v3/Conv_14/BiasAdd/YoloRegion parameters: 
[ INFO ]          num     : 3
[ INFO ]          classes : 80
[ INFO ]          coords  : 4
[ INFO ]          anchors : [10.0, 13.0, 16.0, 30.0, 33.0, 23.0, 30.0, 61.0, 62.0, 45.0, 59.0, 119.0, 116.0, 90.0, 156.0, 198.0, 373.0, 326.0]
[ INFO ] Layer detector/yolo-v3/Conv_22/BiasAdd/YoloRegion parameters: 
[ INFO ]          num     : 3
[ INFO ]          classes : 80
[ INFO ]          coords  : 4
[ INFO ]          anchors : [10.0, 13.0, 16.0, 30.0, 33.0, 23.0, 30.0, 61.0, 62.0, 45.0, 59.0, 119.0, 116.0, 90.0, 156.0, 198.0, 373.0, 326.0]
[ INFO ] Layer detector/yolo-v3/Conv_6/BiasAdd/YoloRegion parameters: 
[ INFO ]          num     : 3
[ INFO ]          classes : 80
[ INFO ]          coords  : 4
[ INFO ]          anchors : [10.0, 13.0, 16.0, 30.0, 33.0, 23.0, 30.0, 61.0, 62.0, 45.0, 59.0, 119.0, 116.0, 90.0, 156.0, 198.0, 373.0, 326.0]
[ INFO ] Layer detector/yolo-v3/Conv_14/BiasAdd/YoloRegion parameters: 
[ INFO ]   

[ INFO ] Layer detector/yolo-v3/Conv_14/BiasAdd/YoloRegion parameters: 
[ INFO ]          num     : 3
[ INFO ]          classes : 80
[ INFO ]          coords  : 4
[ INFO ]          anchors : [10.0, 13.0, 16.0, 30.0, 33.0, 23.0, 30.0, 61.0, 62.0, 45.0, 59.0, 119.0, 116.0, 90.0, 156.0, 198.0, 373.0, 326.0]
[ INFO ] Layer detector/yolo-v3/Conv_22/BiasAdd/YoloRegion parameters: 
[ INFO ]          num     : 3
[ INFO ]          classes : 80
[ INFO ]          coords  : 4
[ INFO ]          anchors : [10.0, 13.0, 16.0, 30.0, 33.0, 23.0, 30.0, 61.0, 62.0, 45.0, 59.0, 119.0, 116.0, 90.0, 156.0, 198.0, 373.0, 326.0]
[ INFO ] Layer detector/yolo-v3/Conv_6/BiasAdd/YoloRegion parameters: 
[ INFO ]          num     : 3
[ INFO ]          classes : 80
[ INFO ]          coords  : 4
[ INFO ]          anchors : [10.0, 13.0, 16.0, 30.0, 33.0, 23.0, 30.0, 61.0, 62.0, 45.0, 59.0, 119.0, 116.0, 90.0, 156.0, 198.0, 373.0, 326.0]
[ INFO ] Layer detector/yolo-v3/Conv_14/BiasAdd/YoloRegion parameters: 
[ INFO ]   

[ INFO ] Layer detector/yolo-v3/Conv_14/BiasAdd/YoloRegion parameters: 
[ INFO ]          num     : 3
[ INFO ]          classes : 80
[ INFO ]          coords  : 4
[ INFO ]          anchors : [10.0, 13.0, 16.0, 30.0, 33.0, 23.0, 30.0, 61.0, 62.0, 45.0, 59.0, 119.0, 116.0, 90.0, 156.0, 198.0, 373.0, 326.0]
[ INFO ] Layer detector/yolo-v3/Conv_22/BiasAdd/YoloRegion parameters: 
[ INFO ]          num     : 3
[ INFO ]          classes : 80
[ INFO ]          coords  : 4
[ INFO ]          anchors : [10.0, 13.0, 16.0, 30.0, 33.0, 23.0, 30.0, 61.0, 62.0, 45.0, 59.0, 119.0, 116.0, 90.0, 156.0, 198.0, 373.0, 326.0]
[ INFO ] Layer detector/yolo-v3/Conv_6/BiasAdd/YoloRegion parameters: 
[ INFO ]          num     : 3
[ INFO ]          classes : 80
[ INFO ]          coords  : 4
[ INFO ]          anchors : [10.0, 13.0, 16.0, 30.0, 33.0, 23.0, 30.0, 61.0, 62.0, 45.0, 59.0, 119.0, 116.0, 90.0, 156.0, 198.0, 373.0, 326.0]
[ INFO ] Layer detector/yolo-v3/Conv_14/BiasAdd/YoloRegion parameters: 
[ INFO ]   

[ INFO ] Layer detector/yolo-v3/Conv_14/BiasAdd/YoloRegion parameters: 
[ INFO ]          num     : 3
[ INFO ]          classes : 80
[ INFO ]          coords  : 4
[ INFO ]          anchors : [10.0, 13.0, 16.0, 30.0, 33.0, 23.0, 30.0, 61.0, 62.0, 45.0, 59.0, 119.0, 116.0, 90.0, 156.0, 198.0, 373.0, 326.0]
[ INFO ] Layer detector/yolo-v3/Conv_22/BiasAdd/YoloRegion parameters: 
[ INFO ]          num     : 3
[ INFO ]          classes : 80
[ INFO ]          coords  : 4
[ INFO ]          anchors : [10.0, 13.0, 16.0, 30.0, 33.0, 23.0, 30.0, 61.0, 62.0, 45.0, 59.0, 119.0, 116.0, 90.0, 156.0, 198.0, 373.0, 326.0]
[ INFO ] Layer detector/yolo-v3/Conv_6/BiasAdd/YoloRegion parameters: 
[ INFO ]          num     : 3
[ INFO ]          classes : 80
[ INFO ]          coords  : 4
[ INFO ]          anchors : [10.0, 13.0, 16.0, 30.0, 33.0, 23.0, 30.0, 61.0, 62.0, 45.0, 59.0, 119.0, 116.0, 90.0, 156.0, 198.0, 373.0, 326.0]
[ INFO ] Layer detector/yolo-v3/Conv_14/BiasAdd/YoloRegion parameters: 
[ INFO ]   

In [ ]:
videoHTML('Yolov3 (Intel CPU)',
           ['results/CPU/cars.mp4']
        )

## Create a Job File 

All the code up to this point has been run within the Jupyter Notebook instance running on a development node based on an Intel® Xeon® Scalable Processor, where the Notebook is allocated a single core. To run inference on the entire video, we need more compute power. We will run the workload on several DevCloud's edge compute nodes. We will send work to the edge compute nodes by submitting jobs into a queue. For each job, we will specify the type of the edge compute server that must be allocated for the job.

To pass the specific variables to the Python code, we will use following arguments:

-m      location of the optimized MobileNet-SSD model's XML
-i      location of the input video
-o      output directory
-d      hardware device type (CPU, GPU, MYRIAD, HDDL or HETERO:FPGA,CPU)
-l      path to the CPU extension library

The job file will be executed directly on the edge compute node.

In [5]:
%%writefile object_detection_job1.sh

ME=`basename $0`

# The default path for the job is your home directory, so we change directory to where the files are.
cd $PBS_O_WORKDIR

# Object detection script writes output to a file inside a directory. We make sure that this directory exists.
# The output directory is the first argument of the bash script
while getopts 'd:f:i:r:n:?' OPTION; do
    case "$OPTION" in
    d)
        DEVICE=$OPTARG
        echo "$ME is using device $OPTARG"
      ;;

    f)
        FP_MODEL=$OPTARG
        echo "$ME is using floating point model $OPTARG"
      ;;

    i)
        INPUT_FILE=$OPTARG
        echo "$ME is using input file $OPTARG"
      ;;
    r)
        RESULTS_BASE=$OPTARG
        echo "$ME is using results base $OPTARG"
      ;;
    n)
        NUM_INFER_REQS=$OPTARG
        echo "$ME is running $OPTARG inference requests"
      ;;
    esac  
done


RESULTS_PATH="${RESULTS_BASE}"
mkdir -p $RESULTS_PATH
echo "$ME is using results path $RESULTS_PATH"

    
# Running the object detection code
SAMPLEPATH=$PBS_O_WORKDIR
python3 object_detection_demo_yolov3_async.py                        -m tensorflow-yolo-v3/${FP_MODEL}/frozen_darknet_yolov3_model.xml \
                                            -i $INPUT_FILE \
                                            -d $DEVICE \
                                            -l /opt/intel/openvino/deployment_tools/inference_engine/lib/intel64/libcpu_extension_avx2.so



Overwriting object_detection_job1.sh


### Job queue submission

Each cell below will submit a job to different edge compute nodes. The output of the cell is the JobID of your job, which you can use to track progress of a job.

Note You can submit all 5 jobs at once or follow one at a time.

After submission, they will go into a queue and run as soon as the requested compute resources become available. (tip: shift+enter will run the cell and automatically move you to the next cell. So you can hit shift+enter multiple times to quickly run multiple cells)

### Intel® CPU
In the cell below, we submit a job to an IEI Tank 870-Q170 edge node with an Intel Core i5-6500TE. The inference workload will run on the CPU.

In [5]:
#Submit job to the queue
job_id_core = !qsub object_detection_job1.sh -l nodes=1:idc001skl:i5-6500te -F "-r results/CPU -d CPU -f FP32 -i $VIDEO -n 2" -N obj_det_core
print(job_id_core[0]) 
#Progress indicators
if job_id_core:
    
    progressIndicator('results/CPU', 'post_progress.txt', "Inferencing", 0, 100)

55252.c003


In [ ]:
videoHTML('Yolov3 (Intel CPU)',
           ['results/CPU/cars.mp4']
        )

### Intel® Core CPU with Intel® GPU

In the cell below, we submit a job to an IEI Tank 870-Q170 edge node with an Intel Core i5-6500TE. The inference workload will run on the Intel® HD Graphics 530 card integrated with the CPU.

In [6]:
#Submit job to the queue
job_id_gpu = !qsub object_detection_job1.sh -l nodes=1:idc001skl:intel-hd-530 -F "-r results/GPU -d GPU -f FP32 -i $VIDEO -n 4" -N obj_det_gpu 
print(job_id_gpu[0]) 
#Progress indicators
if job_id_gpu:


    progressIndicator('results/GPU', 'post_progress.txt', "Inferencing", 0, 100)

58078.c003


In [66]:
videoHTML('Yolov3 (Intel GPU)',
           ['results/GPU/cars.mp4']
        )

### Intel® Neural Compute Stick 2

In the cell below, we submit a job to an IEI Tank 870-Q170 edge node with an Intel Core i5-6500te CPU. The inference workload will run on an Intel Neural Compute Stick 2 installed in this node.

In [65]:
#Submit job to the queue
job_id_ncs2 = !qsub object_detection_job1.sh -l nodes=1:tank-870:i5-6500te:intel-ncs2 -F "-r results/NCS2 -d MYRIAD -f FP16 -i $VIDEO -n 8" -N obj_det_ncs2
print(job_id_ncs2[0]) 
#Progress indicators
if job_id_ncs2:
   
    progressIndicator('results/MYRIAD', 'post_progress.txt', "Inferencing", 0, 100)

57091.c003


In [ ]:
videoHTML('Yolov3 (Intel MYRIAD)',
           ['results/MYRIAD/cars.mp4']
        )

In [9]:
!pbsnodes | grep compnode | awk '{print $3}' | sort | uniq -c

     35 idc001skl,compnode,iei,tank-870,intel-core,i5-6500te,skylake,intel-hd-530,ram8gb,1gbe
     15 idc002mx8,compnode,iei,tank-870,intel-core,i5-6500te,skylake,intel-hd-530,ram8gb,net1gbe,hddl-r,iei-mustang-v100-mx8
     18 idc003a10,compnode,iei,tank-870,intel-core,i5-6500te,skylake,intel-hd-530,ram8gb,net1gbe,hddl-f,iei-mustang-f100-a10
     23 idc004nc2,compnode,iei,tank-870,intel-core,i5-6500te,skylake,intel-hd-530,ram8gb,net1gbe,ncs,intel-ncs2
      5 idc006kbl,compnode,iei,tank-870,intel-core,i5-7500t,kaby-lake,intel-hd-630,ram8gb,net1gbe
     16 idc007xv5,compnode,iei,tank-870,intel-xeon,e3-1268l-v5,skylake,intel-hd-p530,ram32gb,net1gbe
     15 idc008u2g,compnode,up-squared,grove,intel-atom,e3950,apollo-lake,intel-hd-505,ram4gb,net1gbe,ncs,intel-ncs2
      1 idc009jkl,compnode,jwip,intel-core,i5-7500,kaby-lake,intel-hd-630,ram8gb,net1gbe
      1 idc010jal,compnode,jwip,intel-atom,e3950,apollo-lake,intel-hd-505,ram4gb,net1gbe
      2 idc012ros,compnode,iei,tank-870,intel-core,